In [33]:
import os
from collections import Counter
import unidecode
import unicodedata
from unidecode import unidecode_expect_ascii
import gc
import json
import re
from itertools import chain
ROOT = 'ORIGINAL_CSV'
# ROOT = 'CSV'

def get_dodf_type(s, dbg=False, default='integra'):
    s = s.lower()
    if 'sup' in s:
        return 'suplemento'
    elif 'ext' in s or 'etx' in s or 'exr' in s:
        return 'extra'
    elif 'esp' in s:
        return 'especial'
    elif 'int' in s or 'sec' in s:
        return 'integra'
    else:
        if dbg:
            print("SEM CLASSE:", s, '| default: ', default)        
        return default

In [21]:
%%time
# `diretorios`: lista dos DIRETÓRIOS relativos aos DODFs.
# Não há portanto duplicatas, mas ocorre de vários elementos 
# possuírem mesmo prefixo.

diretorios = [i[0] for i in os.walk(ROOT)]
nempty = [i for i in os.walk(ROOT) if (i[1] or i[2])]
empty = [i[0] for i in os.walk(ROOT) if (not i[1] and not i[2])]

print("paths, not-empty, empty")
print(len(diretorios), len(nempty), len(empty), end='\n\n')
del empty, nempty;
gc.collect();

paths, not-empty, empty
17136 9565 7571

CPU times: user 582 ms, sys: 301 ms, total: 884 ms
Wall time: 887 ms


0

In [22]:
diretorios[:4]

['ORIGINAL_CSV',
 'ORIGINAL_CSV/2018',
 'ORIGINAL_CSV/2018/02_Fevereiro',
 'ORIGINAL_CSV/2018/02_Fevereiro/DODF 011 09-02-2018 EDICAO EXTRA']

## Note-se que afinal, as primeiras 3 partes de QUALQUER diretório são dispensáveis para o fim de contagem, servindo apenas para organização

In [23]:
hier = {}
def prefix(x):
    return '/'.joins(x.split('/')[:-1])
for s in diretorios:
    l = s.split('/')
    d = hier
    for p in l[:-1]:
        d = d.get(p, dict())
    d[l[-1]] = dict()
hier[ROOT].keys()

dict_keys(['2018', '2002', '2005', '2003', '2009', '2001', '2006', '2020', '2015', '2013', '2011', '2019', '2012', '2014', '2010', '2017', '2008', '2016', '2004', '2007'])

In [24]:
diretorios_por_ano = {}
for year in hier[ROOT]:
    diretorios_por_ano[year] = [
        i for i in diretorios if year in i and i.count('/') > 2
    ]
diretorios_por_ano['2001'][:10]

['ORIGINAL_CSV/2001/12_Dezembro/DODF 229 - 03-12-2001',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 229 - 03-12-2001/DODF Seção 3 3-12-2001',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 229 - 03-12-2001/DODF Seção 1 3-12-2001',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 229 - 03-12-2001/DODF Seção 2 3-12-2001',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 232 - 06-12-2001',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 232 - 06-12-2001/DODF Seção 1 6-12-2001',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 232 - 06-12-2001/DODF Seção 2 6-12-2001',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 232 - 06-12-2001/DODF Seção 3 6-12-2001',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 243 - 21-12-2001',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 243 - 21-12-2001/DODF Seção 2 21-12-2001']

In [25]:
Counter([i.count('/') for i in diretorios ])

Counter({0: 1, 1: 20, 2: 225, 3: 5489, 4: 11401})

In [26]:
hier = {}
# def normalize_dir(s):
#     s = unidecode_expect_ascii(s)
#     return s.lower()

worthy_dir = [i for i in diretorios if i.count('/') > 2]
# has_deg = []
for i in sorted(
#     map(normalize_dir, worthy_dir),
    worthy_dir,
    key=lambda x: (x.count('/'), re.search(r'\d{4}', x).group())):
    qtd_barra = i.count('/')
#     if 'deg' in i:
#     if '°' in i:
        
#         """"DECISÃO: ignorar esses raríssimos casos"""
#         has_deg.append(i)
#         continue
        
    if qtd_barra == 3:
        if i in hier:
            print("CONFLITO:", i)
        hier[i] = []
#   qtd_barra == 4 ou seja: é um arquivo de uma pasta
#   com várias partes de um DODF
    else:
        hier[ i[:i.rfind('/')] ].append(i.split('/')[-1])
# print("Tem 'deg' no nome:", len(has_deg))
# """Como são muito poucos, serão ignorados"""

In [27]:
# has_deg

In [34]:
Counter({k:get_dodf_type(k) for (k,v) in hier.items()}.values())

Counter({'integra': 4646, 'suplemento': 283, 'extra': 558, 'especial': 2})

In [32]:
_={k:get_dodf_type(k) for (k,v) in hier.items()}
_

{'ORIGINAL_CSV/2001/12_Dezembro/DODF 229 - 03-12-2001': 'integra',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 232 - 06-12-2001': 'integra',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 243 - 21-12-2001': 'integra',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 242 - 20-12-2001': 'integra',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 241 - 19-12-2001': 'integra',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 230 - 04-12-2001': 'integra',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 231 - 05-12-2001': 'integra',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 237 - 13-12-2001': 'integra',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 236 - 12-12-2001': 'integra',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 244 - 26-12-2001': 'integra',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 235 - 11-12-2001': 'integra',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 245 - 27-12-2001': 'integra',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 239 - 17-12-2001': 'integra',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 238 - 14-12-2001': 'integra',
 'ORIGINAL_CSV/2001/12_Dezembro/DODF 240 - 18-12-2001': 'integ

In [35]:
json.dump(hier, open('hierarquia_dodfs.json', 'w'))

In [ ]:
re.findall('.{10}deg', str(hier))

In [ ]:
{k: len(v) for (k,v) in diretorios_por_ano.items()}

In [ ]:
def undo_hier(dic):
    if not dic:
        return []
    else:
        res = []
        for k, v in dic.items():
            all_keys = undo_hier(v)
            res.append(
                [ f'{k}/{i}' for i in all_keys] if all_keys else [k]
            )
        return list(chain(*res))

In [ ]:
hier['csv']['2018'].keys()

In [ ]:
hier['csv']['2017']['04_abril']

In [ ]:
for i in hier['CSV']['2018']['01_Janeiro']:
    print(i, type(i))

In [ ]:
def fix_folder_name(s):
    s = re.sub(r'\ +', ' ', s)
    s = s.replace(' ', _)
    return s

pastas_dodf = [
    '/'.join(i.split('/')[3:] if 'dodf' in i.lower() else i)
     for i in diretorios if i.count('/') > 2
]
pastas_dodf = [unidecode.unidecode(i) for i in pastas_dodf]
pastas_dodf = [i.lower() for i in pastas_dodf]
ctr = Counter(pastas_dodf)

In [ ]:
len(pastas_dodf)

In [ ]:
Counter(pastas_dodf)

In [ ]:
problematicos = [i for i in pastas_dodf if 'dodf' not in i]
len(problematicos)

## Há 136 pastas com nomes problemáticos. Será ignoradas na hora de fazer amostragem, mas consideradas na hora de calcular o tamanha mínimo da amosta

In [ ]:
set(problematicos) == set([i.lower() for i in l])

In [ ]:
# prefixos_diretorios = [
#     '/'.join(i.split('/')[:3]) for i in diretorios
# ]
# Counter(prefixos_diretorios)
# # del diretorios
# # gc.collect()

In [ ]:
pastas_dodf_combarra = [i for i in pastas_dodf if '/' in i]
pastas_dodf_sembarra = [i for i in pastas_dodf if '/' not in i]

print(len(pastas_dodf_sembarra), len(set(pastas_dodf_sembarra)))
print(len(pastas_dodf_combarra), len(set(pastas_dodf_combarra)))
# _=Counter(pastas_dodf_sembarra)

# bad_ones = []
# for i, v in _.items():
#     if v > 1:
#         bad_ones.append((i, v))
# bad_ones

In [ ]:
pastas_dodf_combarra

# Agrupando por pastas, pois há casos com um DODF quebrado em várias seções


In [ ]:
slash_group = {}
for i in pastas_dodf_combarra:
    slash_group[i.split('/')[0]] = []
for i in pastas_dodf_combarra:
    prf = i.split('/')[0]
    slash_group[prf].append(i)
print("Com pastas:", len(slash_group))

In [ ]:
Counter(map(lambda x: x.split('/')[0], pastas_dodf_combarra))

# Mostrando quais nomes (COM BARRAS) são problemáticos. Considerarei-os como 'íntegra'

In [ ]:
for k, v in slash_group.items():
    [get_dodf_type(i, dbg=True) for i in v];

# Mostrando quais nomes (SEM BARRAS) são problemáticos (felizmente, nenhum)

In [ ]:
for i in not_slash:
    get_dodf_type(i)

## Agora, hora de agrupar baseado no tipo de edicao

In [ ]:
by_edition = {}

by_edition_nslash = {
    'suplemento': [],
    'extra': [],
    'especial': [],
    'integra': [],
}
for i in not_slash:
    by_edition_nslash[ get_dodf_type(i) ].append(i)



by_edition_slash = {
    'suplemento': [],
    'extra': [],
    'especial': [],
    'integra': [],
}
for k, v in slash_group.items():
    tipos = [(i, get_dodf_type(i)) for i in v]
    s = set([i[1] for i in tipos])
    for i in s:
        for idx, (fname, tipo) in enumerate(tipos):
            if i == tipo:
                by_edition_slash[tipo].append(v[idx])
                break
for k in by_edition_slash:
    by_edition_slash[k] = sorted(by_edition_slash[k])
    by_edition_nslash[k] = sorted(by_edition_nslash[k])
                
by_edition['COM BARRA'] = by_edition_slash
by_edition['SEM BARRA'] = by_edition_nslash

json.dump(by_edition_nslash, open('nslash.json' ,'w'), 
          indent=4*' ', ensure_ascii=False)
json.dump(by_edition_slash, open('slash.json' ,'w'), 
          indent=4*' ', ensure_ascii=False)

# del by_edition_slash, by_edition_nslash
# gc.collect()

In [ ]:
by_edition_slash

In [ ]:
by_edition['COM BARRA'].keys(), by_edition['SEM BARRA'].keys()

In [ ]:
total = {}
for k in by_edition['COM BARRA']:
    total[k] = 0

for v in by_edition.values():
    for k, lis in v.items():
        total[k] += len(lis)
print("TOTAL: ")
total

# Precisamos ver quais os tipos de DODF cada pasta possui. No caso, são de interesse apenas os DODFs na íntegra.

In [ ]:
_=by_edition_slash['integra']
len(set(_)), len(_)

In [ ]:
_=by_edition_nslash['integra']
len(set(_)), len(_)

In [ ]:
ns_sem_dodf = [
    i for i in by_edition_nslash['integra'] if 'dodf' not in i
]
ns_com_dodf = [
    i for i in by_edition_nslash['integra'] if 'dodf' in i
]
len(ns_sem_dodf), len(ns_com_dodf)

In [ ]:
s_sem_dodf = [
    i for i in by_edition_slash['integra'] if 'dodf' not in i
]
s_com_dodf = [
    i for i in by_edition_slash['integra'] if 'dodf' in i
]
len(s_sem_dodf), len(s_com_dodf)

In [ ]:
for k, v in Counter(by_edition_nslash['integra']).items():
    if v > 1:
        print(k, '|--->', v)

In [ ]:
ns_sem_dodf